In [1]:
import io, json, requests, time, os, os.path, math, urllib
from sys import stdout
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn import linear_model
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime

In [2]:
# returns python object representation of JSON in response
def get_response(symbol, older_than, retries=5):
    url = 'https://api.stocktwits.com/api/2/streams/symbol/%s.json?max=%d' % (symbol, older_than-1)
    for _ in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            return json.loads(response.content)
        elif response.status_code == 429:
            print(response.content)
            return 'limit reached' 
        else:
            return 'Another status' + str(response.status_code)
        time.sleep(1.0)
    # couldn't get response
    return None

In [5]:
# extends the current dataset for a given symbol with more tweets
def get_older_tweets(symbol, num_queries): 
    id_set = set()
    path = './data/%s.json' % symbol
#     if os.path.exists(path):
#         # extending an existing json file
#         with open(path, 'r') as f:
#             data = json.load(f)
#             if len(data) > 0:
#                 older_than = data[-1]['id']
#             else:
#                 older_than = 1000000000000
#     else:
#         # creating a new json file
    #older_than = 1000000000000 # any huge number
    older_than = 181000886
    
    for i in range(num_queries):
        data = []
        url = 'https://api.stocktwits.com/api/2/streams/symbol/%s.json?max=%d' % (symbol, older_than-1)
        content = get_response(symbol, older_than)

        if isinstance(content, str) and content.startswith("Another status"):
            with open("data/log.txt", "a") as l:
                print("ERROR" + content)
                l.write("ERROR" + content + "\n")
                continue
                
        if content == 'limit reached' :
            print("Limit reached at " + str(datetime.datetime.now()))
            print("Wait for an hour...")
            time.sleep(3600)
            print("Wait finished.")
            content = get_response(symbol, older_than)

        #data.extend(content['messages'])
        data = content['messages']        
        older_than = data[-1]['id']
        
        try:
            l = open("data/log.txt", "a")
            l.write("Query " + str(i+1) + ", ")
            l.write(url + ", ")
            l.write("Last_id " + str(older_than) + ", ")
            l.write("id set size: " + str(len(id_set)) + "\n")
        except:
            print("Something went wrong when writing to the log file")
        finally:
            l.close()

        try:
            f = open(path, 'a')
            for d in data:
                id_set.add(d['id'])
                json.dump(d, f)
                f.write(",")
        except:
            print("Something went wrong when writing to the data file")
        finally:
            f.close()
            
#         with open("data/log.txt", "a") as l:
#             l.write("Query " + str(i+1) + ", ")
#             l.write(url + ", ")
#             l.write("Last_id " + str(older_than) + ", ")
#             l.write("id set size: " + str(len(id_set)) + "\n")

#         with open(path, 'a') as f:
#             for d in data:
#                 id_set.add(d['id'])
#                 json.dump(d, f)
#                 f.write(",")

        stdout.write('\rSuccessfully made query %d' % (i+1))
        stdout.flush()
        # sleep to make sure we don't get throttled
        time.sleep(0.5)

#         if i % 200 == 199 and i < num_queries - 1:
#             print("Limit reach at " + str(datetime.datetime.now()))
#             print("Wait for an hour...")
#             time.sleep(3600)
#             print("Wait finished.")

    # write the new data to the JSON file
#     with open(path, 'w') as f:
#         json.dump(data, f)
        
    print()
    print("Done! Number of ids: ", len(id_set))
    return id_set

In [ ]:
# get some data
# apparently a client can only make 200 requests an hour, so we can't get all the data at once
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

# make data directory if needed
if not os.path.exists('./data'):
    os.mkdir('./data')
    
symbols = symbols = ['BTC.X']
number_of_queries = 3000
for symbol in symbols:
    id_set = get_older_tweets(symbol, number_of_queries)   
    
print("Finished reading data at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))           

Starting at 2019-10-22 19:15:56.886618
Successfully made query 68